# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
conda install -c conda-forge gmaps

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\asamu\Anaconda3\envs\PythonData

  added / updated specs:
    - gmaps


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         184 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geojson-2.5.0              |             py_0          15 KB  conda-forge
    gmaps-0.9.0                |             py_0         1.7 MB  conda-forge
    openssl-1.1.1g             |       he774522_0         5.7 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         7.8 MB

The following NEW packages will be INSTALLED:

  geojson            conda-forge/noarch::geo

In [94]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [95]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
# from api_keys import g_key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [96]:
# import csv
file = "../output_data/cities2.csv"

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [97]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [98]:
# produce heatmap layer
vaca_df = pd.read_csv(file)
locations = vaca_df[['Latitude', 'Longitude']].astype(float)
humidity = vaca_df['Humidity (%)'].astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                max_intensity=100,
                                point_radius=6.5)

figure_layout = {
    'width': '600px',
    'height': '450px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# add heatmap layer
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)

# display
fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [99]:
# narrow down cities to fit weather conditions
vaca_clean_df = vaca_df[vaca_df['Max Temp (F)'] <= 80]
vaca_clean_df = vaca_clean_df[vaca_clean_df['Max Temp (F)'] >= 70]
vaca_clean_df = vaca_clean_df[vaca_clean_df['Humidity (%)'] <= 80]
vaca_clean_df = vaca_clean_df[vaca_clean_df['Cloudiness (%)'] <= 20]

# define specs
locations = vaca_clean_df[['Latitude', 'Longitude']].astype(float)
humidity = vaca_clean_df['Humidity (%)'].astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                max_intensity=100,
                                point_radius=6.5)

figure_layout = {
    'width': '600px',
    'height': '450px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# add heat layer
fig_best = gmaps.figure(layout=figure_layout)
fig_best.add_layer(heat_layer)

# display
fig_best

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [100]:
# add 'Hotel Name to new DataFrame'
hotel_df = vaca_clean_df
hotel_df = pd.DataFrame(hotel_df)
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City ID,City,Cloudiness (%),Country,Date,Humidity (%),Latitude,Longitude,Max Temp (F),Wind Speed (mph),Hotel Name
2,2,Jamestown,1,US,1591764762,60,42.10,-79.24,73.40,6.93,
27,27,Carnarvon,0,AU,1591764767,30,-24.87,113.63,75.65,5.26,
75,75,Khor,0,RU,1591764775,56,47.89,134.95,77.95,13.98,
76,76,East London,0,ZA,1591764604,28,-33.02,27.91,73.40,14.99,
81,81,Port Alfred,4,ZA,1591764626,34,-33.59,26.89,75.00,5.99,


In [101]:
# Create List to store hotel names
hotel_list = []

# API Call and parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "types": "lodging",
    "radius": 5000,
    "key": g_key
}

# response
for index, row in hotel_df.iterrows():
    
    city_name = row['City']
    
    lat = row['Latitude']
    lon = row['Longitude']
    params['location'] = (f'{lat},{lon}')
    
    # print results retrieval
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        hotel_list.append(results[0]['name'])
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_list.append("N/A")
              
    print("------------------")

Retrieving Results for Index 2: Jamestown.
------------------
Retrieving Results for Index 27: Carnarvon.
------------------
Retrieving Results for Index 75: Khor.
------------------
Retrieving Results for Index 76: East London.
------------------
Retrieving Results for Index 81: Port Alfred.
------------------
Retrieving Results for Index 98: Cap Malheureux.
------------------
Retrieving Results for Index 115: Bethel.
------------------
Retrieving Results for Index 135: San Jose.
------------------
Retrieving Results for Index 147: Tayoltita.
------------------
Retrieving Results for Index 174: Morro Bay.
------------------
Retrieving Results for Index 250: Laguna.
------------------
Retrieving Results for Index 251: Broome.
------------------
Retrieving Results for Index 254: Cabo San Lucas.
------------------
Retrieving Results for Index 270: Half Moon Bay.
------------------
Retrieving Results for Index 276: Geraldton.
------------------
Retrieving Results for Index 279: Sambava.
-

In [102]:
# remove cities with no hotel within 5000 meters
hotel_df['Hotel Name'] = hotel_list
hotel_df = hotel_df[hotel_df['Hotel Name'] != "N/A"]
hotel_df.head()

,City ID,City,Cloudiness (%),Country,Date,Humidity (%),Latitude,Longitude,Max Temp (F),Wind Speed (mph),Hotel Name
2,2,Jamestown,1,US,1591764762,60,42.10,-79.24,73.40,6.93,Hampton Inn & Suites Jamestown
27,27,Carnarvon,0,AU,1591764767,30,-24.87,113.63,75.65,5.26,Hospitality Carnarvon
75,75,Khor,0,RU,1591764775,56,47.89,134.95,77.95,13.98,Gostinitsa Deksi
76,76,East London,0,ZA,1591764604,28,-33.02,27.91,73.40,14.99,Tu Casa
81,81,Port Alfred,4,ZA,1591764626,34,-33.59,26.89,75.00,5.99,The Halyards Hotel


In [103]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [104]:
# Add marker layer ontop of heat map
locations = hotel_df[['Latitude', 'Longitude']].astype(float)
hotel_name = hotel_df['Hotel Name'].tolist()
city_list = hotel_df['City'].tolist()
country_list = hotel_df['Country'].tolist()

# specify marker layer
markers = gmaps.marker_layer(locations,
                             hover_text = [f'City: {city}' for city in city_list],
                             label = [f'{country}' for country in country_list],
                            info_box_content=[f'Name: {name}' for name in hotel_name])

# add layers
fig_best = gmaps.figure(layout=figure_layout)                             
fig_best.add_layer(markers)
fig_best.add_layer(heat_layer)

# Display figure
fig_best

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…